# Graphing Patches

In [ ]:
from lib.tools import *
from lib.stamp import *
from lib.inject import *
from lib.butler import *
from lib.visual import *
import lsst.geom as geom
from lsst.daf.butler import Butler, Timespan
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy.table import Table, vstack
%matplotlib inline
import lsst.afw.display as afwDisplay
from lsst.source.injection import generate_injection_catalog


In [ ]:
butler = Butler("dp1", collections="LSSTComCam/DP1")

visit_detector_region gives us just one detector, each visit is unique and we don't have duplicates

In [ ]:
band = 'r'
ra = 53.076
dec = -28.11
query = "band.name = :band AND \
    visit_detector_region.region OVERLAPS POINT(:ra, :dec)"
bind = {"band": band, "ra": ra, "dec": dec}
dataset_refs = butler.query_datasets("visit_image",
                                     where=query,
                                     order_by=["visit.timespan.begin"],
                                     bind=bind)

visit.region can tell us all 9 detectors with the same exposure

In [ ]:
band = 'r'
ra = 53.076
dec = -28.11
visit_idx = 1
query = "band.name = :band AND \
    visit.region OVERLAPS POINT(:ra, :dec)"
bind = {"band": band, "ra": ra, "dec": dec}
dataset_refs = butler.query_datasets("visit_image",
                                     where=query,
                                     order_by=["visit.timespan.begin"],
                                     bind=bind)

In [ ]:
band = 'r'
ra = 53.076
dec = -28.11
visit_idx = 1
query = "band.name = :band AND \
    patch.region OVERLAPS POINT(:ra, :dec)"
bind = {"band": band, "ra": ra, "dec": dec}
dataset_patch = butler.query_datasets("template_coadd",
                                     where=query,
                                     bind=bind)

In [ ]:
template = butler.get(dataset_patch[0])

In [ ]:
dir(template)

In [ ]:
#template.getInfo().getMetadata()

In [ ]:
import lsst.skymap
findPatch(lsst.geom.SpherePoint(ra=50,dec=20))

In [ ]:
for d in dataset_refs:
    if d.dataId.get('visit') == dataset_refs[visit_idx].dataId.get('visit'):
        print(d.dataId)

In [ ]:
visit = dataset_refs[visit_idx]

In [ ]:
visit.dataId

In [ ]:
visit_img = butler.get(visit)

In [ ]:
dir(visit_img.image)

In [ ]:
#dir(visit_img)

In [ ]:
type(visit_img.metadata)

In [ ]:
visit_img.visitInfo.boresightRaDec.getDec().asDegrees()

In [ ]:
#dir(visit_img.visitInfo.getBoresightRaDec())

In [ ]:
def pixel_to_degrees(n):
    return 0.2 * n / 3600

In [ ]:
import numpy as np

wcs = template.getWcs()

patch_bbox = template.getBBox()

patch_xmin = patch_bbox.getMinX()
patch_ymin = patch_bbox.getMinY()
patch_height = pixel_to_degrees(template.height)
patch_width = pixel_to_degrees(template.width)

#wcs = deep_coadd.wcs
#print(dir(wcs))

#print(deep_coadd.getBBox().getMin())
wcs.pixelToSky(patch_xmin, patch_ymin) #.getRa().asDegrees()
patch_ra_dec_min = wcs.pixelToSky(patch_xmin, patch_ymin) #.getRa().asDegrees()
patch_ra_min = patch_ra_dec_min.getRa().asDegrees()
patch_dec_min = patch_ra_dec_min.getDec().asDegrees()
patch_width = patch_width / np.cos(np.deg2rad(patch_dec_min))
#print(ra_dec_min) 
print(patch_ra_min, patch_dec_min, patch_height, patch_width)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

ra = visit_img.visitInfo.boresightRaDec.getRa().asDegrees()
dec = visit_img.visitInfo.boresightRaDec.getDec().asDegrees()

bbox = visit_img.getBBox()

# Rectangle parameters
center = (ra, dec)        # (x, y)
width = pixel_to_degrees(bbox.getMaxX())              # Length along x-axis before rotation
height = pixel_to_degrees(bbox.getMaxY())             # Length along y-axis before rotation
angle = visit_img.visitInfo.boresightRotAngle.asDegrees()             # Degrees counter-clockwise

# Convert center to bottom-left corner, accounting for rotation
# The Rectangle class expects the lower-left corner
# So we just specify center as the anchor point for simplicity using transform

# Create figure and axes
fig, ax = plt.subplots()
ax.plot([ra],[dec], 'x')
# Create rectangle
rect = patches.Rectangle(
    (center[0] - width / 2, center[1] - height / 2),  # lower-left corner
    width,
    height,
    angle=-angle,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='blue',
    facecolor='none'
)

patch_rect = patches.Rectangle(
    (patch_ra_min, patch_dec_min),  # lower-left corner
    -patch_width,
    patch_height,
    angle=0,
    rotation_point = 'center',
    linewidth=2,
    edgecolor='red',
    facecolor='none'
)

plt.xlabel("RA [deg]")
plt.ylabel("DEC [deg]")

# Add rectangle to plot
ax.add_patch(rect)
ax.add_patch(patch_rect)

ax.invert_xaxis()

theta_rad = np.deg2rad(-angle)

dx = 0.05 * np.cos(theta_rad)
dy = 0.05 * np.sin(theta_rad)

plt.arrow(ra, dec, -dx, -dy,
          head_width=0.01, head_length=0.01, fc='black', ec='black')# Set axis limits and aspect
plt.arrow(ra, dec, -dy, dx,
          head_width=0.01, head_length=0.01, fc='black', ec='black')# Set axis limits and aspect

plt.annotate('x', xy=(ra - dx/0.7, dec - dy/0.7))
plt.annotate('y', xy=(ra-dy/0.7,dec+dx/0.7))
#ax.set_xlim(0, 10000)
#ax.set_ylim(0, 10000)
ax.set_aspect('equal')

plt.grid(ls=':')

In [ ]:
print(center)

In [ ]:
visit_img.visitInfo.getBoresightRotAngle().asDegrees()

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.Display(frame=1)

In [ ]:
afw_display.mtv(template)

In [ ]:
afw_display.setMaskTransparency(100)

Setting Mask Transparency to 0 shows many colors.
- Yellow: The most saturated parts that exceed an assigned maximum. The Green areas surrounding are the edge-detected bodies drawn around these high points
- Green: Saturated Stars
- Cyan: Cross-talk between amplifiers - When yellow areas have these overflowed electrons, there is a high signal that creates signal interference that spikes at regular intervals on the sides of where they are read.
  - There are 8 amplifiers
- Dark Blue: Detected Sources
- Green Lines: Defects in the CCD

The vertical "bleed trails" come from the incoming electrons overflowing into other pixels, they are vertical due to the physical design of the detector

In [ ]:
visit_wcs = visit_img.getWcs()

In [ ]:
#dir(visit_wcs)

In [ ]:
visit2_wcs = butler.get(dataset_refs[1]).getWcs()
visit_wcs.getRelativeRotationToWcs(visit2_wcs)

In [ ]:
visit_wcs.getSkyOrigin()

In [ ]:
visit_wcs.getPixelScale()

In [ ]:
visit_wcs.getTransform()

In [ ]:
visit_wcs.pixelToSky(x=12., y=13.)

In [ ]:
#visit_wcs.linearizePixelToSky(x=12., y=13.)

In [ ]:
visit_wcs.skyToPixelArray(ra=53,dec=-28)

In [ ]:
visit_wcs.getCdMatrix()

The matrix gives the transformation from pixels to ra,dec in degrees: 
> New system coords = position * (cdmatrix containing deltax and deltay) + "origin coordinate"

In [ ]:
for i in range(20):
    afw_display.dot("o", 1844, 970, size=500, ctype="red")

getFitsMetadata() gives information about any distortions resolved in the final image
- CRPIX: Coordinate Reference Pixel
- CRVAL: Ra(CRVAL1),Dec(CRVAL2) for the origin
- CD1/2_1/2: Elements of the matrix
- A_X_Y: Should be small numbers informing of distortions

In [ ]:
#visit_wcs.getFitsMetadata()

In [ ]:
visit_img.getInfo().getVisitInfo().boresightRotAngle.asDegrees()

In [ ]:
visit_img.getInfo().getVisitInfo()